# facial keypoints detector

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import path as opath
import os
import sys
import seaborn as sns
from PIL import Image
import errno
sns.set(style="whitegrid", color_codes=True)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from utils import get_batches, get_accuracy

Using TensorFlow backend.


In [2]:
# setting 
DATA_PATH = 'data/'
MODEL_PATH = 'models/'
%mkdir -p $MODEL_PATH

TRAIN_PATH = opath.join(DATA_PATH, 'train')
TEST_PATH = opath.join(DATA_PATH, 'test')
VALID_PATH = opath.join(DATA_PATH, 'valid')


## Downloading data

In [ ]:
%mkdir -p $DATA_PATH
%pwd
%cd $DATA_PATH
%pwd
!kg download -c facial-keypoints-detector
%ls
%cd ..
%pwd

## Preprocess data

In [ ]:
def convert_pixels(pix_str):
    return np.array([int(p) for p in pix_str.split(' ')], 'uint8').reshape((48, 48))


def load_data():
    df = pd.read_csv(opath.join(DATA_PATH, 'train.csv'), converters={'Pixels': convert_pixels})
    return df

data = load_data()

In [ ]:
def decode_label(df):
    df = df.copy()
    df.loc[df['Emotion'] == 0, 'Emotion'] = 'anger'
    df.loc[df['Emotion'] == 1, 'Emotion'] = 'disgust'
    df.loc[df['Emotion'] == 2, 'Emotion'] = 'fear'
    df.loc[df['Emotion'] == 3, 'Emotion'] = 'happy'
    df.loc[df['Emotion'] == 4, 'Emotion'] = 'sad'
    df.loc[df['Emotion'] == 5, 'Emotion'] = 'surprise'
    df.loc[df['Emotion'] == 6, 'Emotion'] = 'neutral'
    return df

In [ ]:
decoded_data = decode_label(data)
remove_neutral_data = decoded_data[decoded_data['Emotion'] == 'neutral'].sample(700)
remove_happy_data = decoded_data[decoded_data['Emotion'] == 'happy'].sample(300)
transformed_data = decoded_data.drop((remove_happy_data+remove_neutral_data).index)
test_data = transformed_data.sample(frac=0.1)
transformed_data = transformed_data.drop(test_data.index)
valid_data = transformed_data.sample(frac=0.2)
train_data = transformed_data.drop(valid_data.index)

In [ ]:
def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
            
            
def save_img_files(df, base):
    sub_path = opath.join(DATA_PATH, base)
    for idx, row in df.iterrows():
        category_path = opath.join(sub_path, row['Emotion'])
        mkdir(category_path)
        Image.fromarray(row['Pixels']).save(opath.join(category_path, '{}.png'.format(idx)), 'png')

In [ ]:
save_img_files(test_data, 'test')
save_img_files(valid_data, 'valid')
save_img_files(train_data, 'train')

## Vanilla model

In [3]:
def get_bench_model():
    model = Sequential()
    model.add(Conv2D(16, 3, activation='relu', padding='same', input_shape=(3, 48, 48)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(32, 3, activation='relu', padding='same',))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, 3, activation='relu', padding='same',))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    return model

bench_model = get_bench_model()
bench_model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 48, 48)        448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 24, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 24, 24)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 12, 12)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 6, 6)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
__________

In [4]:
bench_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=90)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator()

In [6]:
batch_size = 32
bench_weights_path = opath.join(MODEL_PATH, 'bench.weights.best.hdf5')

checkpointer = ModelCheckpoint(filepath=bench_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen, (48, 48))
valid_batches = get_batches(VALID_PATH, valid_datagen, (48, 48))

bench_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//batch_size, 
                          validation_data=valid_batches, validation_steps=valid_batches.samples//batch_size,
                          callbacks=[checkpointer],  epochs=30)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/30
72/71 [==============================] - 4s 54ms/step - loss: 1.9463 - acc: 0.1465 - val_loss: 1.9301 - val_acc: 0.2010
Epoch 2/30
72/71 [==============================] - 2s 29ms/step - loss: 1.9117 - acc: 0.2141 - val_loss: 1.9615 - val_acc: 0.1748
Epoch 3/30
72/71 [==============================] - 2s 30ms/step - loss: 1.8338 - acc: 0.2690 - val_loss: 1.7652 - val_acc: 0.2885
Epoch 4/30
72/71 [==============================] - 2s 30ms/step - loss: 1.7470 - acc: 0.3163 - val_loss: 1.7001 - val_acc: 0.3322
Epoch 5/30
72/71 [==============================] - 2s 31ms/step - loss: 1.6830 - acc: 0.3488 - val_loss: 1.6403 - val_acc: 0.3636
Epoch 6/30
72/71 [==============================] - 2s 30ms/step - loss: 1.6235 - acc: 0.3711 - val_loss: 1.5647 - val_acc: 0.4126
Epoch 7/30
72/71 [==============================] - 2s 30ms/step - loss: 1.5877 - acc: 0.3860 - val_loss: 1.5260 - val_acc: 0.4790


72/71 [==============================] - 2s 31ms/step - loss: 1.1946 - acc: 0.5574 - val_loss: 1.0510 - val_acc: 0.6189
Epoch 29/30
72/71 [==============================] - 2s 31ms/step - loss: 1.1709 - acc: 0.5654 - val_loss: 1.0369 - val_acc: 0.6241
Epoch 30/30
72/71 [==============================] - 2s 30ms/step - loss: 1.1570 - acc: 0.5727 - val_loss: 1.0719 - val_acc: 0.6101


In [7]:
bench_model.load_weights(bench_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (48, 48))

print "benchmark model get accuary {}%".format(get_accuracy(bench_model, test_batches))

Found 318 images belonging to 7 classes.
benchmark model get accuary 0.522012578991%


## VGG16 finetune

In [6]:
from utils.vgg16 import get_model
vgg_model = get_model()

vgg_model.pop()
for layer in vgg_model.layers:
    layer.trainable = False

vgg_model.add(Dense(7, activation='softmax'))

vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_40 (ZeroPaddi (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 64, 114, 114)      0         
__________

In [11]:
vgg_model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [13]:
top_weights_path = opath.join(MODEL_PATH, 'vgg_tune.weights.best.hdf5')
checkpointer = ModelCheckpoint(filepath=top_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen)
valid_batches = get_batches(VALID_PATH, valid_datagen)

vgg_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//train_batches.batch_size,
                          validation_data=valid_batches, validation_steps=valid_batches.samples//valid_batches.batch_size,
                          callbacks=[checkpointer],  epochs=30, verbose=1)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/30
72/71 [==============================] - 13s 179ms/step - loss: 2.7046 - acc: 0.2348 - val_loss: 1.8039 - val_acc: 0.2640
Epoch 2/30
72/71 [==============================] - 10s 136ms/step - loss: 2.2808 - acc: 0.3088 - val_loss: 1.4828 - val_acc: 0.4650
Epoch 3/30
72/71 [==============================] - 10s 136ms/step - loss: 2.0207 - acc: 0.3813 - val_loss: 1.3413 - val_acc: 0.5367
Epoch 4/30
72/71 [==============================] - 10s 136ms/step - loss: 1.9427 - acc: 0.3953 - val_loss: 1.1815 - val_acc: 0.5542
Epoch 5/30
72/71 [==============================] - 9s 128ms/step - loss: 1.8136 - acc: 0.4310 - val_loss: 1.5151 - val_acc: 0.4318
Epoch 6/30
72/71 [==============================] - 10s 137ms/step - loss: 1.8700 - acc: 0.4147 - val_loss: 1.1353 - val_acc: 0.5944
Epoch 7/30
72/71 [==============================] - 11s 156ms/step - loss: 1.7033 - acc: 0.4717 - val_loss: 1.2785 - val

In [16]:
vgg_model.load_weights(top_weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (224, 224))

print "vgg16 model finetune top layer get accuary {}%".format(get_accuracy(vgg_model, test_batches))

Found 318 images belonging to 7 classes.
vgg16 model finetune top layer get accuary 0.635220125411%


## vgg16 fintune all dense layer

In [19]:
for idx, layer in enumerate(vgg_model.layers):
    print idx, layer.name

0 lambda_4
1 zero_padding2d_40
2 conv2d_40
3 zero_padding2d_41
4 conv2d_41
5 max_pooling2d_16
6 zero_padding2d_42
7 conv2d_42
8 zero_padding2d_43
9 conv2d_43
10 max_pooling2d_17
11 zero_padding2d_44
12 conv2d_44
13 zero_padding2d_45
14 conv2d_45
15 zero_padding2d_46
16 conv2d_46
17 max_pooling2d_18
18 zero_padding2d_47
19 conv2d_47
20 zero_padding2d_48
21 conv2d_48
22 zero_padding2d_49
23 conv2d_49
24 max_pooling2d_19
25 zero_padding2d_50
26 conv2d_50
27 zero_padding2d_51
28 conv2d_51
29 zero_padding2d_52
30 conv2d_52
31 max_pooling2d_20
32 flatten_4
33 dense_12
34 dropout_7
35 dense_13
36 dropout_8
37 dense_15


In [20]:
for layer in vgg_model.layers[:33]:
    layer.trainable = False
for layer in vgg_model.layers[33:]:
    layer.trainable = True

In [21]:
from keras.optimizers import SGD
sgd = SGD(0.001, 0.9, 0.0001, True)
vgg_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
dense_weights_path = opath.join(MODEL_PATH, 'vgg_tune_dense.weights.best.hdf5')
checkpointer = ModelCheckpoint(filepath=dense_weights_path, verbose=1, save_best_only=True)

train_batches = get_batches(TRAIN_PATH, train_datagen)
valid_batches = get_batches(VALID_PATH, valid_datagen)

vgg_model.fit_generator(train_batches, steps_per_epoch=train_batches.samples//train_batches.batch_size,
                          validation_data=valid_batches, validation_steps=valid_batches.samples//valid_batches.batch_size,
                          callbacks=[checkpointer],  epochs=5, verbose=1)


Found 2288 images belonging to 7 classes.
Found 572 images belonging to 7 classes.
Epoch 1/5
72/71 [==============================] - 14s 199ms/step - loss: 1.8245 - acc: 0.4218 - val_loss: 0.9365 - val_acc: 0.6521
Epoch 2/5
72/71 [==============================] - 11s 157ms/step - loss: 1.1461 - acc: 0.6001 - val_loss: 1.0541 - val_acc: 0.5839
Epoch 3/5
72/71 [==============================] - 12s 168ms/step - loss: 0.8879 - acc: 0.6739 - val_loss: 0.8550 - val_acc: 0.6818
Epoch 4/5
72/71 [==============================] - 16s 227ms/step - loss: 0.7730 - acc: 0.7234 - val_loss: 0.7144 - val_acc: 0.7483
Epoch 5/5
72/71 [==============================] - 15s 210ms/step - loss: 0.6304 - acc: 0.7753 - val_loss: 0.6983 - val_acc: 0.7657


In [23]:
vgg_model.load_weights(weights_path)
test_batches = get_batches(TEST_PATH, test_datagen, (224, 224))

print "vgg16 model finetune dense layer get accuary {}%".format(get_accuracy(vgg_model, test_batches))

Found 318 images belonging to 7 classes.
vgg16 model finetune dense layer get accuary 0.738993709942%
